This Jupyter notebook is a test to determine how OpenAi's GPT-3.5 Turbo responds when classifying emojis, and how prompting can affect that. This information was not used in the project, but could be useful for future developments.

In [6]:
## Load libraries
import pandas as pd
import matplotlib.pyplot as plt
from langchain.llms.openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.chains import SequentialChain
import openai
from getpass import getpass
import os
import warnings
import pandas as pd
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


warnings.filterwarnings("ignore")

In [2]:
nsc_posts = pd.read_csv('Instagram Concatenated Data/Instagram Posts.csv')
nsc_comments = pd.read_csv('Instagram Concatenated Data/Instagram Comments.csv')

In [3]:
nsc_posts

,timestamp,hashtags/0,type,isSponsored,ownerUsername,productType,caption,locationId,url,locationName,ownerId,displayUrl,likesCount,commentsCount,id,ownerFullName,hashtags/1,hashtags/2,hashtags/3,hashtags/4
0,2023-09-27T15:12:55.000Z,NaN,Sidecar,False,nashvillesc,NaN,Last night we announced our Team Award winners...,1.006379e+14,https://www.instagram.com/p/CxsypZLuFSB/,GEODIS Park,3824089459,https://instagram.fbko4-2.fna.fbcdn.net/v/t51....,2143,29,3201156182017856641,Nashville Soccer Club,NaN,NaN,NaN,NaN
1,2023-09-23T21:54:09.000Z,NaN,Image,False,nashvillesc,NaN,👀👀,1.062201e+14,https://www.instagram.com/p/CxjNYjFPD15/,"Nashville, Tennessee",3824089459,https://scontent-ord5-1.cdninstagram.com/v/t51...,2501,52,3198459021904067961,Nashville Soccer Club,NaN,NaN,NaN,NaN
2,2023-09-24T04:32:23.000Z,NaN,Image,False,nashvillesc,NaN,A point on the road,1.101577e+14,https://www.instagram.com/p/Cxj69RnsgfS/,PayPal Park,3824089459,https://scontent-mia3-1.cdninstagram.com/v/t51...,2113,49,3198659457868498898,Nashville Soccer Club,NaN,NaN,NaN,NaN
3,2023-09-23T19:07:48.000Z,SJvNSH.,Video,False,nashvillesc,clips,Come out strong & keep the momentum going 💪\r\...,NaN,https://www.instagram.com/p/Cxi572EvbIA/,NaN,3824089459,https://instagram.fesb9-1.fna.fbcdn.net/v/t51....,827,3,3198373486548333056,Nashville Soccer Club,NaN,NaN,NaN,NaN
4,2023-09-24T00:06:24.000Z,SJvNSH,Image,False,nashvillesc,NaN,Keeping the lights on late tonight 💡 \r\n\r\n#...,1.101577e+14,https://www.instagram.com/p/CxjchNXsEPZ/,PayPal Park,3824089459,https://scontent-lax3-2.cdninstagram.com/v/t51...,2947,20,3198525587764298713,Nashville Soccer Club,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,2022-09-10T13:56:54.000Z,NaN,Sidecar,False,nashvillesc,NaN,4️⃣ wins in a row. Let's make it 5️⃣ today,1.006379e+14,https://www.instagram.com/p/CiVCSRuMytE/,GEODIS Park,3824089459,https://scontent-atl3-1.cdninstagram.com/v/t51...,1636,4,2924253585119324996,Nashville Soccer Club,NaN,NaN,NaN,NaN
880,2022-09-09T01:46:08.000Z,NaN,Sidecar,False,nashvillesc,NaN,All N it together,1.006379e+14,https://www.instagram.com/p/CiRJ3LqANFX/,GEODIS Park,3824089459,https://scontent-iad3-1.cdninstagram.com/v/t51...,1716,5,2923161007645839703,Nashville Soccer Club,NaN,NaN,NaN,NaN
881,2022-09-06T18:08:31.000Z,NaN,Sidecar,False,nashvillesc,NaN,⚡️🫡,1.062201e+14,https://www.instagram.com/p/CiLL5rMgFxg/,"Nashville, Tennessee",3824089459,https://instagram.fhex6-1.fna.fbcdn.net/v/t51....,1655,11,2921481125182332000,Nashville Soccer Club,NaN,NaN,NaN,NaN
882,2022-09-10T01:59:23.000Z,NaN,Video,False,nashvillesc,clips,"Respect the question, @lex_mmwheel19 😂",1.062201e+14,https://www.instagram.com/p/CiTwG6zAMAL/,"Nashville, Tennessee",3824089459,https://scontent-mia3-1.cdninstagram.com/v/t51...,3282,50,2923892164775690251,Nashville Soccer Club,NaN,NaN,NaN,NaN


In [10]:
nsc_comments.tail(50)

,text,id,likesCount,timestamp,postUrl,ownerUsername
19332,It’s nice and all but we have to win the playoffs,NaN,NaN,2022-10-02T21:44:53.000Z,https://www.instagram.com/p/CjOestMr4fA/,onpak.31
19333,Let’s gooooo boys ❤️👑,NaN,NaN,2022-10-02T21:50:45.000Z,https://www.instagram.com/p/CjOestMr4fA/,edasilvaofficial
19334,👏🏿👏🏿👏🏿,NaN,NaN,2022-10-02T21:52:52.000Z,https://www.instagram.com/p/CjOestMr4fA/,cowboytroyglobal
19335,Woohooo!!! Let’s gooooo!!!,NaN,NaN,2022-10-02T21:55:32.000Z,https://www.instagram.com/p/CjOestMr4fA/,emmyjean315
19336,Turn me up @parker_rody,NaN,NaN,2022-10-02T21:56:10.000Z,https://www.instagram.com/p/CjOestMr4fA/,andersonivey_
19337,Congratulations gentlemen! Have a blast in the...,NaN,NaN,2022-10-02T21:57:34.000Z,https://www.instagram.com/p/CjOestMr4fA/,chandlerjohns
19338,💙💛👏👏👏,NaN,NaN,2022-10-02T22:00:16.000Z,https://www.instagram.com/p/CjOestMr4fA/,dsseale
19339,Let’s GOOOOOO,NaN,NaN,2022-10-02T22:02:03.000Z,https://www.instagram.com/p/CjOestMr4fA/,ryanctierney
19340,@nashvillesc @yrsdoow,NaN,NaN,2022-10-02T22:02:04.000Z,https://www.instagram.com/p/CjOestMr4fA/,rileyx28
19341,@rileyx28 yessssssss 😩🙏🏼🙏🏼🙏🏼🙏🏼,NaN,NaN,2022-10-02T22:02:22.000Z,https://www.instagram.com/p/CjOestMr4fA/,rysdoow


In [25]:
OPENAI_API_KEY = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")
llm = OpenAI(temperature = 0.7, model="gpt-3.5-turbo-instruct")

········


In [52]:
claps = nsc_comments.loc[19355, "text"]
claps

'👏🏼🎉👏🏼🎉👏🏼🎉👏🏼'

In [84]:
## Sentiment Analysis Chain
prompt_template2 = '''
The following text is a comment from an Instagram post made by Nashville SC. Please analyze the text and assign it a sentiment score to determine whether it is positive or negative. This sentiment should be on a continuous scale of -1 to 1, where -1 is the most negative, 0 is the most neutral, and 1 is the most postive. Round the score to 2 decimal places.
Comment: {comment}
Please note that the clown emoji (🤡) is usually used in a mocking way.
'''

example_prompt2 = PromptTemplate(input_variables = ["comment"], template = prompt_template2)

aspect_sentiment_chain = LLMChain(llm = llm, prompt = example_prompt2, output_key = "sentiment")

In [95]:
aspect_sentiment_chain({"comment": '"I think we have a chance" 🤡'})

{'comment': '"I think we have a chance" 🤡',
 'sentiment': '\nSentiment Score: -0.5'}